# Application de Masques aux Images COVID-19

Ce notebook démontre comment appliquer les masques aux images de radiographies pulmonaires du dataset COVID-19.

**Objectifs :**
- Explorer la structure des données d'images et de masques
- Développer des fonctions pour appliquer différents types de masques
- Traiter les images par lot
- Visualiser et sauvegarder les résultats

**Dataset :** COVID-19 Radiography Database
- **COVID** : 7,232 images
- **Normal** : 20,384 images  
- **Lung_Opacity** : 12,024 images
- **Viral Pneumonia** : 2,690 images

## 1. Import des Bibliothèques Requises

In [ ]:
# Import des bibliothèques nécessaires
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from PIL import Image
import seaborn as sns

# Configuration pour les graphiques
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

# Ajouter le chemin src au PYTHONPATH pour importer nos modules
sys.path.append('../src')

print("✅ Bibliothèques importées avec succès !")
print(f"📍 Répertoire de travail : {os.getcwd()}")
print(f"🐍 Version OpenCV : {cv2.__version__}")
print(f"📊 Version NumPy : {np.__version__}")

## 2. Configuration des Chemins et Exploration du Dataset

Définition des chemins vers les données et exploration de la structure du dataset.

In [ ]:
# Configuration des chemins
data_path = Path("../data")
dataset_path = data_path / "COVID-19_Radiography_Dataset"

# Vérification de l'existence du dataset
if not dataset_path.exists():
    print("❌ Dataset non trouvé !")
    print(f"Chemin recherché : {dataset_path.absolute()}")
else:
    print("✅ Dataset trouvé !")
    print(f"📁 Chemin : {dataset_path.absolute()}")

# Exploration de la structure
categories = ["COVID", "Normal", "Lung_Opacity", "Viral Pneumonia"]
stats = {}

print("\n📊 Structure du dataset :")
print("-" * 50)

for category in categories:
    category_path = dataset_path / category
    if category_path.exists():
        images_path = category_path / "images"
        masks_path = category_path / "masks"
        
        n_images = len(list(images_path.glob("*.png"))) if images_path.exists() else 0
        n_masks = len(list(masks_path.glob("*.png"))) if masks_path.exists() else 0
        
        stats[category] = {"images": n_images, "masks": n_masks}
        
        print(f"📂 {category}:")
        print(f"   🖼️  Images : {n_images}")
        print(f"   🎭 Masques : {n_masks}")
        print(f"   ✅ Correspondance : {'Oui' if n_images == n_masks else 'Non'}")
        print()

# Statistiques globales
total_images = sum([stats[cat]["images"] for cat in stats])
total_masks = sum([stats[cat]["masks"] for cat in stats])

print(f"📈 TOTAL : {total_images:,} images et {total_masks:,} masques")
print(f"🎯 Dataset prêt pour l'analyse : {'✅' if total_images == total_masks else '❌'}")

## 3. Fonctions d'Application de Masques

Implémentation des différentes méthodes d'application de masques.

In [ ]:
class MaskApplicator:
    """Classe pour appliquer différents types de masques aux images radiographiques"""
    
    def __init__(self):
        self.methods = ["overlay", "multiply", "extract"]
    
    def apply_mask(self, image_path, mask_path, method="overlay", alpha=0.7):
        """
        Applique un masque à une image selon la méthode spécifiée
        
        Args:
            image_path: Chemin vers l'image
            mask_path: Chemin vers le masque
            method: Méthode d'application ("overlay", "multiply", "extract")
            alpha: Transparence pour la méthode overlay (0.0 à 1.0)
        
        Returns:
            tuple: (image_with_mask, original_image, mask)
        """
        # Chargement des images
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        
        if image is None or mask is None:
            raise ValueError(f"Impossible de charger l'image ou le masque")
        
        # Redimensionner le masque si nécessaire
        if image.shape != mask.shape:
            mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
        
        # Normaliser les valeurs
        image_norm = image.astype(np.float32) / 255.0
        mask_norm = mask.astype(np.float32) / 255.0
        
        # Application selon la méthode
        if method == "overlay":
            # Superposition avec transparence
            result = cv2.addWeighted(image, 1-alpha, mask, alpha, 0)
        
        elif method == "multiply":
            # Multiplication pour extraction des zones
            result = (image_norm * mask_norm * 255).astype(np.uint8)
        
        elif method == "extract":
            # Extraction pure (garde seulement les zones du masque)
            result = np.where(mask_norm > 0.5, image, 0).astype(np.uint8)
        
        else:
            raise ValueError(f"Méthode '{method}' non supportée. Utilisez: {self.methods}")
        
        return result, image, mask
    
    def get_image_mask_pairs(self, category):
        """Récupère les paires image/masque pour une catégorie"""
        category_path = dataset_path / category
        images_path = category_path / "images"
        masks_path = category_path / "masks"
        
        if not images_path.exists() or not masks_path.exists():
            return []
        
        pairs = []
        for img_file in images_path.glob("*.png"):
            mask_file = masks_path / img_file.name
            if mask_file.exists():
                pairs.append((img_file, mask_file))
        
        return pairs

# Création de l'instance
mask_applicator = MaskApplicator()
print("✅ Classe MaskApplicator créée avec succès !")
print(f"🔧 Méthodes disponibles : {mask_applicator.methods}")

## 4. Visualisation d'Échantillons

Affichage d'exemples d'images avec leurs masques pour chaque catégorie.

In [ ]:
def visualize_samples(n_samples=1):
    """Affiche des échantillons pour chaque catégorie"""
    
    fig, axes = plt.subplots(len(categories), 4, figsize=(16, 4*len(categories)))
    fig.suptitle("Échantillons par Catégorie : Image Originale - Masque - Overlay - Extract", 
                 fontsize=16, fontweight='bold')
    
    for i, category in enumerate(categories):
        pairs = mask_applicator.get_image_mask_pairs(category)
        
        if pairs:
            # Prendre le premier échantillon
            img_path, mask_path = pairs[0]
            
            try:
                # Application des masques
                overlay_result, original, mask = mask_applicator.apply_mask(
                    img_path, mask_path, method="overlay", alpha=0.5)
                extract_result, _, _ = mask_applicator.apply_mask(
                    img_path, mask_path, method="extract")
                
                # Affichage
                axes[i, 0].imshow(original, cmap='gray')
                axes[i, 0].set_title(f'{category}\nImage Originale')
                axes[i, 0].axis('off')
                
                axes[i, 1].imshow(mask, cmap='gray')
                axes[i, 1].set_title('Masque')
                axes[i, 1].axis('off')
                
                axes[i, 2].imshow(overlay_result, cmap='gray')
                axes[i, 2].set_title('Overlay (α=0.5)')
                axes[i, 2].axis('off')
                
                axes[i, 3].imshow(extract_result, cmap='gray')
                axes[i, 3].set_title('Extract')
                axes[i, 3].axis('off')
                
            except Exception as e:
                for j in range(4):
                    axes[i, j].text(0.5, 0.5, f'Erreur: {str(e)}', 
                                   ha='center', va='center', transform=axes[i, j].transAxes)
                    axes[i, j].axis('off')
        else:
            for j in range(4):
                axes[i, j].text(0.5, 0.5, 'Aucune donnée', 
                               ha='center', va='center', transform=axes[i, j].transAxes)
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Affichage des échantillons
print("🖼️ Affichage des échantillons...")
visualize_samples()

## 5. Comparaison des Méthodes d'Application

Comparaison détaillée des différentes méthodes d'application de masques sur un même échantillon.

In [ ]:
def compare_methods(category="COVID", sample_idx=0):
    """Compare les différentes méthodes d'application de masques"""
    
    pairs = mask_applicator.get_image_mask_pairs(category)
    if not pairs or sample_idx >= len(pairs):
        print(f"❌ Pas d'échantillon disponible pour {category} à l'index {sample_idx}")
        return
    
    img_path, mask_path = pairs[sample_idx]
    
    # Test de différents paramètres alpha pour overlay
    alphas = [0.3, 0.5, 0.7]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle(f'Comparaison des Méthodes - {category} (Échantillon {sample_idx})', 
                 fontsize=16, fontweight='bold')
    
    try:
        # Images originales
        original = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        
        # Première ligne : Image originale et masque
        axes[0, 0].imshow(original, cmap='gray')
        axes[0, 0].set_title('Image Originale')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(mask, cmap='gray')
        axes[0, 1].set_title('Masque')
        axes[0, 1].axis('off')
        
        # Overlays avec différents alphas
        for i, alpha in enumerate(alphas):
            overlay_result, _, _ = mask_applicator.apply_mask(
                img_path, mask_path, method="overlay", alpha=alpha)
            axes[0, 2+i].imshow(overlay_result, cmap='gray')
            axes[0, 2+i].set_title(f'Overlay α={alpha}')
            axes[0, 2+i].axis('off')
        
        # Deuxième ligne : Méthodes multiply et extract
        multiply_result, _, _ = mask_applicator.apply_mask(
            img_path, mask_path, method="multiply")
        axes[1, 0].imshow(multiply_result, cmap='gray')
        axes[1, 0].set_title('Multiply')
        axes[1, 0].axis('off')
        
        extract_result, _, _ = mask_applicator.apply_mask(
            img_path, mask_path, method="extract")
        axes[1, 1].imshow(extract_result, cmap='gray')
        axes[1, 1].set_title('Extract')
        axes[1, 1].axis('off')
        
        # Masque coloré pour mieux voir les zones
        mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        axes[1, 2].imshow(cv2.cvtColor(mask_colored, cv2.COLOR_BGR_RGB))
        axes[1, 2].set_title('Masque Coloré')
        axes[1, 2].axis('off')
        
        # Histogramme des pixels du masque
        axes[1, 3].hist(mask.flatten(), bins=50, alpha=0.7, color='blue')
        axes[1, 3].set_title('Histogramme du Masque')
        axes[1, 3].set_xlabel('Intensité')
        axes[1, 3].set_ylabel('Fréquence')
        
    except Exception as e:
        print(f"❌ Erreur lors de la comparaison : {e}")
    
    plt.tight_layout()
    plt.show()

# Test avec un échantillon COVID
print("🔬 Comparaison des méthodes sur un échantillon COVID...")
compare_methods("COVID", 0)

## 6. Traitement par Lots

Application de masques sur plusieurs images et sauvegarde des résultats.

In [ ]:
def batch_process_masks(category, method="extract", max_samples=10, save_results=False):
    """
    Traite plusieurs images d'une catégorie avec application de masques
    
    Args:
        category: Catégorie à traiter
        method: Méthode d'application du masque
        max_samples: Nombre maximum d'échantillons à traiter
        save_results: Si True, sauvegarde les résultats
    """
    print(f"🔄 Traitement par lots : {category} - Méthode : {method}")
    
    pairs = mask_applicator.get_image_mask_pairs(category)
    if not pairs:
        print(f"❌ Aucune paire image/masque trouvée pour {category}")
        return
    
    # Limiter le nombre d'échantillons
    pairs = pairs[:max_samples]
    
    results = []
    output_dir = Path("../results") / f"{category}_{method}"
    
    if save_results:
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"📁 Dossier de sortie : {output_dir}")
    
    # Barre de progression
    for i, (img_path, mask_path) in enumerate(tqdm(pairs, desc=f"Traitement {category}")):
        try:
            # Application du masque
            result, original, mask = mask_applicator.apply_mask(
                img_path, mask_path, method=method)
            
            # Sauvegarde si demandée
            if save_results:
                output_path = output_dir / f"{img_path.stem}_{method}.png"
                cv2.imwrite(str(output_path), result)
            
            # Statistiques
            result_stats = {
                'filename': img_path.name,
                'original_mean': np.mean(original),
                'original_std': np.std(original),
                'mask_coverage': np.sum(mask > 128) / mask.size * 100,
                'result_mean': np.mean(result),
                'result_std': np.std(result)
            }
            results.append(result_stats)
            
        except Exception as e:
            print(f"❌ Erreur avec {img_path.name}: {e}")
            continue
    
    # Analyse des résultats
    if results:
        df_results = pd.DataFrame(results)
        
        print(f"\n📊 Résultats du traitement par lots ({len(results)} images)")
        print("-" * 60)
        print(f"Couverture moyenne du masque : {df_results['mask_coverage'].mean():.1f}%")
        print(f"Intensité moyenne originale : {df_results['original_mean'].mean():.1f}")
        print(f"Intensité moyenne résultat : {df_results['result_mean'].mean():.1f}")
        
        if save_results:
            # Sauvegarde des statistiques
            stats_path = output_dir / "statistics.csv"
            df_results.to_csv(stats_path, index=False)
            print(f"📈 Statistiques sauvées : {stats_path}")
        
        return df_results
    
    return None

# Test du traitement par lots
print("🚀 Test du traitement par lots...")
results_covid = batch_process_masks("COVID", method="extract", max_samples=5, save_results=True)

## 7. Analyse Statistique des Masques

Analyse des propriétés statistiques des masques pour chaque catégorie.

In [ ]:
def analyze_mask_statistics(max_samples_per_category=20):
    """Analyse statistique des masques pour toutes les catégories"""
    
    print("📊 Analyse statistique des masques...")
    
    all_stats = []
    
    for category in categories:
        print(f"\n🔍 Analyse de {category}...")
        pairs = mask_applicator.get_image_mask_pairs(category)
        
        if not pairs:
            continue
        
        # Limiter le nombre d'échantillons pour l'analyse
        pairs = pairs[:max_samples_per_category]
        
        category_stats = []
        
        for img_path, mask_path in tqdm(pairs[:max_samples_per_category], 
                                       desc=f"Analyse {category}"):
            try:
                # Chargement du masque
                mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
                if mask is None:
                    continue
                
                # Calcul des statistiques
                mask_binary = (mask > 128).astype(np.uint8)
                coverage = np.sum(mask_binary) / mask_binary.size * 100
                
                # Contours pour calculer la forme
                contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                area = 0
                perimeter = 0
                if contours:
                    largest_contour = max(contours, key=cv2.contourArea)
                    area = cv2.contourArea(largest_contour)
                    perimeter = cv2.arcLength(largest_contour, True)
                
                stats = {
                    'category': category,
                    'filename': img_path.name,
                    'coverage_percent': coverage,
                    'mask_mean': np.mean(mask),
                    'mask_std': np.std(mask),
                    'area': area,
                    'perimeter': perimeter,
                    'compactness': 4 * np.pi * area / (perimeter**2) if perimeter > 0 else 0
                }
                
                category_stats.append(stats)
                all_stats.append(stats)
                
            except Exception as e:
                print(f"❌ Erreur avec {img_path.name}: {e}")
                continue
        
        # Statistiques pour cette catégorie
        if category_stats:
            df_cat = pd.DataFrame(category_stats)
            print(f"   📈 Couverture moyenne : {df_cat['coverage_percent'].mean():.1f}%")
            print(f"   📏 Aire moyenne : {df_cat['area'].mean():.0f} pixels")
            print(f"   🎯 Compacité moyenne : {df_cat['compactness'].mean():.3f}")
    
    # Analyse globale
    if all_stats:
        df_all = pd.DataFrame(all_stats)
        
        # Graphiques de comparaison
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Analyse Statistique des Masques par Catégorie', fontsize=16, fontweight='bold')
        
        # Distribution de la couverture
        for category in categories:
            cat_data = df_all[df_all['category'] == category]
            if not cat_data.empty:
                axes[0, 0].hist(cat_data['coverage_percent'], alpha=0.7, label=category, bins=20)
        axes[0, 0].set_title('Distribution de la Couverture (%)')
        axes[0, 0].set_xlabel('Pourcentage de couverture')
        axes[0, 0].set_ylabel('Fréquence')
        axes[0, 0].legend()
        
        # Boxplot de la couverture par catégorie
        df_all.boxplot(column='coverage_percent', by='category', ax=axes[0, 1])
        axes[0, 1].set_title('Couverture par Catégorie')
        axes[0, 1].set_ylabel('Pourcentage de couverture')
        
        # Distribution de l'aire
        for category in categories:
            cat_data = df_all[df_all['category'] == category]
            if not cat_data.empty:
                axes[1, 0].hist(cat_data['area'], alpha=0.7, label=category, bins=20)
        axes[1, 0].set_title('Distribution de l\'Aire')
        axes[1, 0].set_xlabel('Aire (pixels)')
        axes[1, 0].set_ylabel('Fréquence')
        axes[1, 0].legend()
        
        # Relation aire vs compacité
        for category in categories:
            cat_data = df_all[df_all['category'] == category]
            if not cat_data.empty:
                axes[1, 1].scatter(cat_data['area'], cat_data['compactness'], 
                                 alpha=0.7, label=category, s=30)
        axes[1, 1].set_title('Aire vs Compacité')
        axes[1, 1].set_xlabel('Aire (pixels)')
        axes[1, 1].set_ylabel('Compacité')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
        
        # Tableau récapitulatif
        summary = df_all.groupby('category').agg({
            'coverage_percent': ['mean', 'std'],
            'area': ['mean', 'std'],
            'compactness': ['mean', 'std']
        }).round(2)
        
        print("\\n📋 Tableau récapitulatif :")
        print(summary)
        
        return df_all
    
    return None

# Lancement de l'analyse
mask_stats = analyze_mask_statistics(max_samples_per_category=15)

## 8. Conclusion et Export des Résultats

Résumé de l'analyse et export des résultats pour utilisation ultérieure.